In [1]:
from utils.prompt import experience_extraction_prompt
from utils import get_response, get_history_posts

history_items, username_dict = get_history_posts(17000)


2025-12-21 14:54:25.899 | INFO     | utils.message_utils:get_history_posts:69 - 加载帖子缓存 16600 条，用户缓存 315 条
2025-12-21 14:54:25.900 | INFO     | utils.message_utils:get_history_posts:70 - 准备获取历史消息，limit=17000，before=None
2025-12-21 14:54:25.901 | INFO     | utils.message_utils:get_history_posts:113 - 请求API获取消息，page_limit=100，next_before=None
2025-12-21 14:54:26.808 | INFO     | utils.message_utils:get_history_posts:179 - 检测到数据与缓存重合，尝试从缓存补齐剩余 16900 条
2025-12-21 14:54:27.136 | INFO     | utils.message_utils:get_history_posts:223 - 最终返回消息数量：16600


In [2]:
from typing import List, Dict, Any

def keep_admin_nearby(posts: List[Dict[str, Any]], window_each_side: int = 20) -> List[Dict[str, Any]]:
    """
    保留 isPosterAdmin=True 的帖子，以及其前后各 window_each_side 条（去重后保持原顺序）。
    默认前后各20条 => “附近40条”(上下文)。
    """
    n = len(posts)
    keep_idx = set()

    admin_indices = [i for i, p in enumerate(posts) if p.get("isPosterAdmin", False)]
    for i in admin_indices:
        start = max(0, i - window_each_side)
        end = min(n, i + window_each_side + 1)
        keep_idx.update(range(start, end))

    return [posts[i] for i in range(n) if i in keep_idx]
posts = keep_admin_nearby(history_items, window_each_side=20)

In [3]:
from utils import history_list_to_text

big_text = history_list_to_text(posts, username_dict)
to_summary_text = experience_extraction_prompt.replace("<<MESSAGE_CONTENT>>", big_text)
model = "gemini-3-flash-preview"
summary = get_response(to_summary_text, model=model)

2025-12-21 14:54:36.122 | INFO     | utils.agent:get_response:19 - 正在使用模型 gemini-3-flash-preview 生成总结...


2025-12-21 14:54:58.862 | INFO     | utils.agent:get_response:49 - 模型生成总结完成。


In [4]:
with open("zhaoge_trading_method.md", "w", encoding="utf-8") as f:
    f.write(summary)